# Pandas Profissional: Transformação de dados com Métodos Encadeados em um Estudo de Caso

**Pandas Method Chaining** - Formação do indicador 12c do Plano Nacional de Educação (PNE)

por [Aldéryck Albuquerque](https://www.linkedin.com/in/alderyck/) - 2023/09 - [github.com/derycck](https://github.com/derycck/Pandas/blob/main/pandas_method_chaining/Ex001/Exemplo_001.ipynb)

`#pandas #chaining #python #data #dataWrangling #dataAnalysis #dataEngineer`

A adoção da técnica method chaining no Pandas revoluciona a eficiência da análise de dados. Ao encadear métodos, cada passo de manipulação é executado de forma contínua e intuitiva, eliminando variáveis intermediárias e simplificando o código. Isso não só aprimora a legibilidade, mas também potencializa a produtividade, permitindo análises complexas em poucas linhas. Abraçar o "method chaining" é abraçar o futuro da análise de dados ágil e poderosa.

Ao finalizar este tutorial você terá capacidade de:
- Compreender as vantagens associadas a essa abordagem de métodos encadeado
- Dominar algumas técnicas baseada em ações e regras de negócio
- Aplicar essas técnicas para a construção do indicador 12c do PNE

## Roteiro

- **Método convencional ensinado em cursos introdutórios**
- **Pandas Method Chaining**
    - **Tutorial Intensivo**
    - **Estudo de Caso**
- **Mais fontes de aprendizado**

---

Indicador do Plano Nacional de Educação (PNE)

**12C-Participação do segmento público na expansão de matrículas de graduação**

Fórmula: "Expansão Pública" / "Expansão Total"

Numerador: Variação das matrículas em cursos de graduação de IES públicas 

Denominador: Variação total das matrículas em cursos de graduação. 

Fórmula com variáveis: (QT_PUBLICA - QT_PUBLICA_2013) / (QT_TOTAL - QT_TOTAL_2013) * 100

Período: 2014~2021

Granularidade: Ano & Município

Dicionário de dados
- NU_ANO_CENSO: Ano do Censo
- FK_MUNICIPIO_CODIGO: Código do município de acordo com o IBGE
- TP_NIVEL_ACADEMICO: Nível acadêmico do curso, "Graduação" corresponde ao valor 1
- TP_CATEGORIA_ADMINISTRATIVA: Categoria "Pública" corresponde aos valores [1,2,3,7]
- QT_MAT: Quantidade de matrículas no curso

Características dos dados:
- Originado da tabela de Cursos do Censo de Educação Superior, obtido no [site do INEP](https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/censo-da-educacao-superior).
- Gradunaridade de "Ano & Cursos". Cada linha contém dados de um Curso específico
- Contém linhas com valores vazios em FK_MUNICIPIO_CODIGO

`A granularidade refere-se ao nível de detalhe ou à menor unidade de informação representada por cada linha da tabela`

In [340]:
import pandas as pd

In [341]:
file_path = 'src_censo_educ_superior_cursos.csv.xz'
df = pd.read_csv(file_path)

Trabalharemos com uma base de dados de 1,8 milhões de linhas do Censo de Educação Superior

In [342]:
print(df.shape)
df.head(3)

(1831958, 5)


,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,TP_NIVEL_ACADEMICO,TP_CATEGORIA_ADMINISTRATIVA,QT_MAT
0,2021,NaN,1,1,0.0
1,2021,NaN,1,1,0.0
2,2021,NaN,1,1,0.0


Ao final do tutorial você será de entender e replicar a transformação abaixo

**Versão Final**

In [415]:
(df
 .dropna(subset='FK_MUNICIPIO_CODIGO', axis=0)
 .rename(columns={'NU_ANO_CENSO': 'ANO'})
 .assign(QT_MAT=lambda x:x['QT_MAT'].fillna(0))
 .astype({**{k:'uint16' for k in ['ANO','QT_MAT']}, 
          'FK_MUNICIPIO_CODIGO':'uint32',
          'TP_NIVEL_ACADEMICO':'uint8'})
 .query('TP_NIVEL_ACADEMICO==1 and ANO>=2013')
 .assign(PUBLICA="PUBLICA_SIM")
 .assign(PUBLICA=lambda x: (x['PUBLICA']
                            .where(x['TP_CATEGORIA_ADMINISTRATIVA'].isin([1,2,3,7]), 
                                   "PUBLICA_NAO")))
 .groupby(["ANO", "FK_MUNICIPIO_CODIGO","PUBLICA"])["QT_MAT"]
 .sum().unstack().reset_index().rename_axis(None, axis=1)
 .fillna(0)
 .assign(QT_TOTAL=lambda x:x['PUBLICA_SIM']+x['PUBLICA_NAO'])
 # 2013 cols
 .pipe(lambda w:(
    w.merge(w.loc[lambda y: y['ANO'].isin([2013]), 
                  ['FK_MUNICIPIO_CODIGO', 'PUBLICA_NAO', 'PUBLICA_SIM']]
             .rename(columns={'PUBLICA_SIM': 'QT_PUBLICA_2013'})
             .assign(QT_TOTAL_2013=lambda x:x['QT_PUBLICA_2013'] + x['PUBLICA_NAO'])
             
             .reindex(['FK_MUNICIPIO_CODIGO', 'QT_TOTAL_2013', 'QT_PUBLICA_2013'], 
                      axis=1), 
            on='FK_MUNICIPIO_CODIGO', how='left')))
 .rename(columns={'PUBLICA_SIM': 'QT_PUBLICA'})
 .query('ANO>=2014')
 .drop(columns=['PUBLICA_NAO'])
 .fillna(0)
 .assign(PP_ATENDIMENTO_IND = lambda x:(
     ((x['QT_PUBLICA']-x['QT_PUBLICA_2013']) / 
      (x['QT_TOTAL']-x['QT_TOTAL_2013'])
     )*100).round(2))
 .astype({**{k:'uint32' for k in ['QT_PUBLICA', 'QT_TOTAL',
                                  'QT_TOTAL_2013', 'QT_PUBLICA_2013']}})
 .assign(INDICADOR="12C")
 .reset_index(drop=True)
)

,ANO,FK_MUNICIPIO_CODIGO,QT_PUBLICA,QT_TOTAL,QT_TOTAL_2013,QT_PUBLICA_2013,PP_ATENDIMENTO_IND,INDICADOR
0,2014,1100023,434,4931,4628,333,33.33,12C
1,2014,1100049,1114,7482,6748,1156,-5.72,12C
2,2014,1100056,0,664,467,0,0.00,12C
3,2014,1100064,575,799,866,592,25.37,12C
4,2014,1100106,777,1029,928,726,50.50,12C
...,...,...,...,...,...,...,...,...
17638,2021,5221809,1106,1106,743,743,100.00,12C
17639,2021,5221858,126,9768,3216,0,1.92,12C
17640,2021,5222005,0,372,0,0,0.00,12C
17641,2021,5222054,0,1,18,0,-0.00,12C


## Método convencional ensinado em Cursos Introdutórios

Primeiramente vamos revisar os conhecimentos básicos que costumam ser ensinados em cursos introdutórios de Pandas, enquanto construímos o dataset para o tal indicador.

In [344]:
df_select = df[['NU_ANO_CENSO', 'FK_MUNICIPIO_CODIGO', 'TP_NIVEL_ACADEMICO', 
                'TP_CATEGORIA_ADMINISTRATIVA', 'QT_MAT']]
df_select

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,TP_NIVEL_ACADEMICO,TP_CATEGORIA_ADMINISTRATIVA,QT_MAT
0,2021,NaN,1,1,0.0
1,2021,NaN,1,1,0.0
2,2021,NaN,1,1,0.0
3,2021,NaN,1,1,0.0
4,2021,NaN,1,1,0.0
...,...,...,...,...,...
1831953,2009,4111803.0,1,2,130.0
1831954,2009,4111803.0,1,2,137.0
1831955,2009,4111803.0,1,2,278.0
1831956,2009,4111803.0,1,2,339.0


remover vazios

In [345]:
df_clean_01 = df_select.dropna(subset='FK_MUNICIPIO_CODIGO', axis=0)
df_clean_01

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,TP_NIVEL_ACADEMICO,TP_CATEGORIA_ADMINISTRATIVA,QT_MAT
7948,2021,5300108.0,1,1,0.0
7949,2021,5300108.0,1,1,0.0
7950,2021,5300108.0,1,1,498.0
7951,2021,5300108.0,1,1,0.0
7952,2021,5300108.0,1,1,0.0
...,...,...,...,...,...
1831953,2009,4111803.0,1,2,130.0
1831954,2009,4111803.0,1,2,137.0
1831955,2009,4111803.0,1,2,278.0
1831956,2009,4111803.0,1,2,339.0


Preencher valor vazio na coluna QT_MAT

In [346]:
df_clean_01['QT_MAT'] = df_clean_01['QT_MAT'].fillna(0)
df_clean_01

C:\Users\pci7\AppData\Local\Temp\ipykernel_11104\761293374.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean_01['QT_MAT'] = df_clean_01['QT_MAT'].fillna(0)


,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,TP_NIVEL_ACADEMICO,TP_CATEGORIA_ADMINISTRATIVA,QT_MAT
7948,2021,5300108.0,1,1,0.0
7949,2021,5300108.0,1,1,0.0
7950,2021,5300108.0,1,1,498.0
7951,2021,5300108.0,1,1,0.0
7952,2021,5300108.0,1,1,0.0
...,...,...,...,...,...
1831953,2009,4111803.0,1,2,130.0
1831954,2009,4111803.0,1,2,137.0
1831955,2009,4111803.0,1,2,278.0
1831956,2009,4111803.0,1,2,339.0


In [347]:
df_clean_01.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1799032 entries, 7948 to 1831957
Data columns (total 5 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   NU_ANO_CENSO                 int64  
 1   FK_MUNICIPIO_CODIGO          float64
 2   TP_NIVEL_ACADEMICO           int64  
 3   TP_CATEGORIA_ADMINISTRATIVA  int64  
 4   QT_MAT                       float64
dtypes: float64(2), int64(3)
memory usage: 82.4 MB


Corrigir dtype

In [348]:
df_clean_02 = df_clean_01.copy()

In [349]:
df_clean_02['NU_ANO_CENSO'] = df_clean_02['NU_ANO_CENSO'].astype('uint16')
df_clean_02['QT_MAT'] = df_clean_02['QT_MAT'].astype('uint16')
df_clean_02['FK_MUNICIPIO_CODIGO'] = df_clean_02['FK_MUNICIPIO_CODIGO'].astype('uint32')
df_clean_02['TP_NIVEL_ACADEMICO'] = df_clean_02['TP_NIVEL_ACADEMICO'].astype('uint8')
df_clean_02.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1799032 entries, 7948 to 1831957
Data columns (total 5 columns):
 #   Column                       Dtype 
---  ------                       ----- 
 0   NU_ANO_CENSO                 uint16
 1   FK_MUNICIPIO_CODIGO          uint32
 2   TP_NIVEL_ACADEMICO           uint8 
 3   TP_CATEGORIA_ADMINISTRATIVA  int64 
 4   QT_MAT                       uint16
dtypes: int64(1), uint16(2), uint32(1), uint8(1)
memory usage: 42.9 MB


Selecionar apenas TP_NIVEL_ACADEMICO == 1 e NU_ANO_CENSO>=2013

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html

In [350]:
mask = ((df_clean_02['TP_NIVEL_ACADEMICO'].isin([1])) & (df_clean_02['NU_ANO_CENSO']>=2013))
df_clean_03 = df_clean_02.loc[mask, :]
df_clean_03

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,TP_NIVEL_ACADEMICO,TP_CATEGORIA_ADMINISTRATIVA,QT_MAT
7948,2021,5300108,1,1,0
7949,2021,5300108,1,1,0
7950,2021,5300108,1,1,498
7951,2021,5300108,1,1,0
7952,2021,5300108,1,1,0
...,...,...,...,...,...
1620476,2013,2307304,1,1,200
1620477,2013,3550605,1,2,101
1620478,2013,2706703,1,5,107
1620479,2013,2706703,1,5,141


criar coluna PUBLIC

In [351]:
mask = df_clean_03['TP_CATEGORIA_ADMINISTRATIVA'].isin([1,2,3,7])
print(mask.sum(), 'linhas a serem alteradas')

df_clean_03.loc[mask, 'PUBLICA'] = "PUBLICA_SIM"
df_clean_03.loc[~mask, 'PUBLICA'] = 'PUBLICA_NAO'
df_clean_03

126860 linhas a serem alteradas


C:\Users\pci7\AppData\Local\Temp\ipykernel_11104\2284344125.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean_03.loc[mask, 'PUBLICA'] = "PUBLICA_SIM"


,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,TP_NIVEL_ACADEMICO,TP_CATEGORIA_ADMINISTRATIVA,QT_MAT,PUBLICA
7948,2021,5300108,1,1,0,PUBLICA_SIM
7949,2021,5300108,1,1,0,PUBLICA_SIM
7950,2021,5300108,1,1,498,PUBLICA_SIM
7951,2021,5300108,1,1,0,PUBLICA_SIM
7952,2021,5300108,1,1,0,PUBLICA_SIM
...,...,...,...,...,...,...
1620476,2013,2307304,1,1,200,PUBLICA_SIM
1620477,2013,3550605,1,2,101,PUBLICA_SIM
1620478,2013,2706703,1,5,107,PUBLICA_NAO
1620479,2013,2706703,1,5,141,PUBLICA_NAO


In [352]:
df_clean_03['PUBLICA'].value_counts()

PUBLICA
PUBLICA_NAO    1463418
PUBLICA_SIM     126860
Name: count, dtype: int64

remover colunas desnecessárias

In [353]:
df_clean_04 = df_clean_03.drop(columns=['TP_NIVEL_ACADEMICO', 'TP_CATEGORIA_ADMINISTRATIVA']).copy()
df_clean_04.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1590278 entries, 7948 to 1620480
Data columns (total 4 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   NU_ANO_CENSO         1590278 non-null  uint16
 1   FK_MUNICIPIO_CODIGO  1590278 non-null  uint32
 2   QT_MAT               1590278 non-null  uint16
 3   PUBLICA              1590278 non-null  object
dtypes: object(1), uint16(2), uint32(1)
memory usage: 36.4+ MB


In [354]:
df_clean_04

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,QT_MAT,PUBLICA
7948,2021,5300108,0,PUBLICA_SIM
7949,2021,5300108,0,PUBLICA_SIM
7950,2021,5300108,498,PUBLICA_SIM
7951,2021,5300108,0,PUBLICA_SIM
7952,2021,5300108,0,PUBLICA_SIM
...,...,...,...,...
1620476,2013,2307304,200,PUBLICA_SIM
1620477,2013,3550605,101,PUBLICA_SIM
1620478,2013,2706703,107,PUBLICA_NAO
1620479,2013,2706703,141,PUBLICA_NAO


transformar valores da coluna "PUBLICA" em colunas independentes

In [355]:
mask = df_clean_04['PUBLICA']=='PUBLICA_SIM'
df_publica_sim = df_clean_04.loc[mask,['NU_ANO_CENSO','FK_MUNICIPIO_CODIGO','QT_MAT', 'PUBLICA']]
df_publica_sim['PUBLICA_SIM'] = df_publica_sim['QT_MAT']
df_publica_sim.drop('PUBLICA', axis=1,inplace=True)
df_publica_sim

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,QT_MAT,PUBLICA_SIM
7948,2021,5300108,0,0
7949,2021,5300108,0,0
7950,2021,5300108,498,498
7951,2021,5300108,0,0
7952,2021,5300108,0,0
...,...,...,...,...
1620473,2013,2307304,80,80
1620474,2013,2307304,134,134
1620475,2013,2307304,291,291
1620476,2013,2307304,200,200


In [356]:
df_publica_sim = df_publica_sim.groupby(['NU_ANO_CENSO', 'FK_MUNICIPIO_CODIGO']).agg({'PUBLICA_SIM':sum})
df_publica_sim

PUBLICA_SIM
NU_ANO_CENSO FK_MUNICIPIO_CODIGO             
2013         1100023                      333
             1100049                     1156
             1100064                      592
             1100106                      726
             1100122                     1092
...                                       ...
2021         5221601                      760
             5221700                       59
             5221809                     1106
             5221858                      126
             5300108                    45158

[11504 rows x 1 columns]

In [357]:
df_publica_sim = df_publica_sim.reset_index()
df_publica_sim = df_publica_sim.fillna(0)
df_publica_sim

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,PUBLICA_SIM
0,2013,1100023,333
1,2013,1100049,1156
2,2013,1100064,592
3,2013,1100106,726
4,2013,1100122,1092
...,...,...,...
11499,2021,5221601,760
11500,2021,5221700,59
11501,2021,5221809,1106
11502,2021,5221858,126


In [358]:
mask = df_clean_04['PUBLICA']=='PUBLICA_NAO'
df_publica_nao = df_clean_04.loc[mask,['NU_ANO_CENSO', 'FK_MUNICIPIO_CODIGO','QT_MAT', 'PUBLICA']]
df_publica_nao['PUBLICA_NAO'] = df_publica_nao['QT_MAT']
df_publica_nao.drop('PUBLICA', axis=1,inplace=True)
df_publica_nao = df_publica_nao.groupby(['NU_ANO_CENSO','FK_MUNICIPIO_CODIGO']).agg({'PUBLICA_NAO':sum})
df_publica_nao = df_publica_nao.reset_index()
df_publica_nao = df_publica_nao.fillna(0)
df_publica_nao

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,PUBLICA_NAO
0,2013,1100023,4295
1,2013,1100049,5592
2,2013,1100056,467
3,2013,1100064,274
4,2013,1100106,202
...,...,...,...
16185,2021,5221700,10
16186,2021,5221858,9642
16187,2021,5222005,372
16188,2021,5222054,1


In [359]:
df_clean_05 = df_clean_04[['NU_ANO_CENSO','FK_MUNICIPIO_CODIGO']].drop_duplicates()
df_clean_05

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO
7948,2021,5300108
10747,2021,5200050
10779,2021,5200100
10793,2021,5200134
10855,2021,5200308
...,...,...
1620239,2013,2929206
1620364,2013,1702554
1620372,2013,4218202
1620401,2013,1717503


In [360]:
df_clean_05 = df_clean_05.merge(df_publica_sim,
                                on=['NU_ANO_CENSO','FK_MUNICIPIO_CODIGO'],
                                how='left')
df_clean_05

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,PUBLICA_SIM
0,2021,5300108,45158.0
1,2021,5200050,NaN
2,2021,5200100,NaN
3,2021,5200134,NaN
4,2021,5200308,147.0
...,...,...,...
19505,2013,2929206,42.0
19506,2013,1702554,NaN
19507,2013,4218202,NaN
19508,2013,1717503,34.0


In [361]:
df_clean_05 = df_clean_05.merge(df_publica_nao,
                                on=['NU_ANO_CENSO','FK_MUNICIPIO_CODIGO'],
                                how='left')
df_clean_05

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,PUBLICA_SIM,PUBLICA_NAO
0,2021,5300108,45158.0,176323.0
1,2021,5200050,NaN,11.0
2,2021,5200100,NaN,6.0
3,2021,5200134,NaN,243.0
4,2021,5200308,147.0,537.0
...,...,...,...,...
19505,2013,2929206,42.0,NaN
19506,2013,1702554,NaN,649.0
19507,2013,4218202,NaN,242.0
19508,2013,1717503,34.0,NaN


In [362]:
df_clean_05 = df_clean_05.fillna(0)
df_clean_05

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,PUBLICA_SIM,PUBLICA_NAO
0,2021,5300108,45158.0,176323.0
1,2021,5200050,0.0,11.0
2,2021,5200100,0.0,6.0
3,2021,5200134,0.0,243.0
4,2021,5200308,147.0,537.0
...,...,...,...,...
19505,2013,2929206,42.0,0.0
19506,2013,1702554,0.0,649.0
19507,2013,4218202,0.0,242.0
19508,2013,1717503,34.0,0.0


</br>

cria coluna QT_TOTAL

In [363]:
df_clean_06 = df_clean_05.copy()
df_clean_06['QT_TOTAL'] = df_clean_06['PUBLICA_SIM'] + df_clean_06['PUBLICA_NAO']
df_clean_06 = df_clean_06.fillna(0)
df_clean_06

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,PUBLICA_SIM,PUBLICA_NAO,QT_TOTAL
0,2021,5300108,45158.0,176323.0,221481.0
1,2021,5200050,0.0,11.0,11.0
2,2021,5200100,0.0,6.0,6.0
3,2021,5200134,0.0,243.0,243.0
4,2021,5200308,147.0,537.0,684.0
...,...,...,...,...,...
19505,2013,2929206,42.0,0.0,42.0
19506,2013,1702554,0.0,649.0,649.0
19507,2013,4218202,0.0,242.0,242.0
19508,2013,1717503,34.0,0.0,34.0


Cria dataframe de visão 2013

In [364]:
mask = df_clean_06['NU_ANO_CENSO'].isin([2013])
df_2013 = df_clean_06.loc[mask, ['FK_MUNICIPIO_CODIGO', 'PUBLICA_NAO', 'PUBLICA_SIM']]
df_2013

,FK_MUNICIPIO_CODIGO,PUBLICA_NAO,PUBLICA_SIM
17643,5100201,626.0,47.0
17644,5100300,568.0,704.0
17645,5101704,88.0,2014.0
17646,5101803,3935.0,1240.0
17647,5103205,1939.0,364.0
...,...,...,...
19505,2929206,0.0,42.0
19506,1702554,649.0,0.0
19507,4218202,242.0,0.0
19508,1717503,0.0,34.0


In [365]:
df_2013.rename(columns={'PUBLICA_SIM': 'QT_PUBLICA_2013'}, inplace=True)
df_2013

,FK_MUNICIPIO_CODIGO,PUBLICA_NAO,QT_PUBLICA_2013
17643,5100201,626.0,47.0
17644,5100300,568.0,704.0
17645,5101704,88.0,2014.0
17646,5101803,3935.0,1240.0
17647,5103205,1939.0,364.0
...,...,...,...
19505,2929206,0.0,42.0
19506,1702554,649.0,0.0
19507,4218202,242.0,0.0
19508,1717503,0.0,34.0


In [366]:
df_2013['QT_TOTAL_2013']=df_2013['QT_PUBLICA_2013'] + df_2013['PUBLICA_NAO']
df_2013

,FK_MUNICIPIO_CODIGO,PUBLICA_NAO,QT_PUBLICA_2013,QT_TOTAL_2013
17643,5100201,626.0,47.0,673.0
17644,5100300,568.0,704.0,1272.0
17645,5101704,88.0,2014.0,2102.0
17646,5101803,3935.0,1240.0,5175.0
17647,5103205,1939.0,364.0,2303.0
...,...,...,...,...
19505,2929206,0.0,42.0,42.0
19506,1702554,649.0,0.0,649.0
19507,4218202,242.0,0.0,242.0
19508,1717503,0.0,34.0,34.0


In [367]:
df_2013 = df_2013[['FK_MUNICIPIO_CODIGO', 'QT_TOTAL_2013', 'QT_PUBLICA_2013']].copy()
df_2013

,FK_MUNICIPIO_CODIGO,QT_TOTAL_2013,QT_PUBLICA_2013
17643,5100201,673.0,47.0
17644,5100300,1272.0,704.0
17645,5101704,2102.0,2014.0
17646,5101803,5175.0,1240.0
17647,5103205,2303.0,364.0
...,...,...,...
19505,2929206,42.0,42.0
19506,1702554,649.0,0.0
19507,4218202,242.0,0.0
19508,1717503,34.0,34.0


mesclar dataframe geral com dataframe 2013

In [368]:
df_clean_07 = df_clean_06.merge(df_2013, on='FK_MUNICIPIO_CODIGO', how='left')
df_clean_07

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,PUBLICA_SIM,PUBLICA_NAO,QT_TOTAL,QT_TOTAL_2013,QT_PUBLICA_2013
0,2021,5300108,45158.0,176323.0,221481.0,203821.0,32582.0
1,2021,5200050,0.0,11.0,11.0,NaN,NaN
2,2021,5200100,0.0,6.0,6.0,NaN,NaN
3,2021,5200134,0.0,243.0,243.0,110.0,0.0
4,2021,5200308,147.0,537.0,684.0,386.0,116.0
...,...,...,...,...,...,...,...
19505,2013,2929206,42.0,0.0,42.0,42.0,42.0
19506,2013,1702554,0.0,649.0,649.0,649.0,0.0
19507,2013,4218202,0.0,242.0,242.0,242.0,0.0
19508,2013,1717503,34.0,0.0,34.0,34.0,34.0


Remover anos anteriores a 2014

In [369]:
mask = df_clean_07['NU_ANO_CENSO']>=2014
df_clean_08 = df_clean_07.loc[mask, :].copy()
df_clean_08

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,PUBLICA_SIM,PUBLICA_NAO,QT_TOTAL,QT_TOTAL_2013,QT_PUBLICA_2013
0,2021,5300108,45158.0,176323.0,221481.0,203821.0,32582.0
1,2021,5200050,0.0,11.0,11.0,NaN,NaN
2,2021,5200100,0.0,6.0,6.0,NaN,NaN
3,2021,5200134,0.0,243.0,243.0,110.0,0.0
4,2021,5200308,147.0,537.0,684.0,386.0,116.0
...,...,...,...,...,...,...,...
17638,2014,2929206,295.0,0.0,295.0,42.0,42.0
17639,2014,4218202,0.0,253.0,253.0,242.0,0.0
17640,2014,1717503,13.0,0.0,13.0,34.0,34.0
17641,2014,3540002,425.0,0.0,425.0,389.0,389.0


Renomear coluna PUBLICA_SIM e preencher valores vazios

In [370]:
df_clean_08.rename(columns={'PUBLICA_SIM': 'QT_PUBLICA'}, inplace=True)
df_clean_08
 

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,QT_PUBLICA,PUBLICA_NAO,QT_TOTAL,QT_TOTAL_2013,QT_PUBLICA_2013
0,2021,5300108,45158.0,176323.0,221481.0,203821.0,32582.0
1,2021,5200050,0.0,11.0,11.0,NaN,NaN
2,2021,5200100,0.0,6.0,6.0,NaN,NaN
3,2021,5200134,0.0,243.0,243.0,110.0,0.0
4,2021,5200308,147.0,537.0,684.0,386.0,116.0
...,...,...,...,...,...,...,...
17638,2014,2929206,295.0,0.0,295.0,42.0,42.0
17639,2014,4218202,0.0,253.0,253.0,242.0,0.0
17640,2014,1717503,13.0,0.0,13.0,34.0,34.0
17641,2014,3540002,425.0,0.0,425.0,389.0,389.0


In [371]:
df_clean_08 = df_clean_08.drop(columns=['PUBLICA_NAO'])
df_clean_08

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,QT_PUBLICA,QT_TOTAL,QT_TOTAL_2013,QT_PUBLICA_2013
0,2021,5300108,45158.0,221481.0,203821.0,32582.0
1,2021,5200050,0.0,11.0,NaN,NaN
2,2021,5200100,0.0,6.0,NaN,NaN
3,2021,5200134,0.0,243.0,110.0,0.0
4,2021,5200308,147.0,684.0,386.0,116.0
...,...,...,...,...,...,...
17638,2014,2929206,295.0,295.0,42.0,42.0
17639,2014,4218202,0.0,253.0,242.0,0.0
17640,2014,1717503,13.0,13.0,34.0,34.0
17641,2014,3540002,425.0,425.0,389.0,389.0


In [372]:
df_clean_08 = df_clean_08.fillna(0)
df_clean_08

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,QT_PUBLICA,QT_TOTAL,QT_TOTAL_2013,QT_PUBLICA_2013
0,2021,5300108,45158.0,221481.0,203821.0,32582.0
1,2021,5200050,0.0,11.0,0.0,0.0
2,2021,5200100,0.0,6.0,0.0,0.0
3,2021,5200134,0.0,243.0,110.0,0.0
4,2021,5200308,147.0,684.0,386.0,116.0
...,...,...,...,...,...,...
17638,2014,2929206,295.0,295.0,42.0,42.0
17639,2014,4218202,0.0,253.0,242.0,0.0
17640,2014,1717503,13.0,13.0,34.0,34.0
17641,2014,3540002,425.0,425.0,389.0,389.0


Criar coluna PP_ATENDIMENTO_IND

In [373]:
df_clean_09 = df_clean_08.copy()
df_clean_09['PP_ATENDIMENTO_IND']=(df_clean_08['QT_PUBLICA']-df_clean_08['QT_PUBLICA_2013']) /  (df_clean_08['QT_TOTAL']-df_clean_08['QT_TOTAL_2013'])
df_clean_09

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,QT_PUBLICA,QT_TOTAL,QT_TOTAL_2013,QT_PUBLICA_2013,PP_ATENDIMENTO_IND
0,2021,5300108,45158.0,221481.0,203821.0,32582.0,0.712118
1,2021,5200050,0.0,11.0,0.0,0.0,0.000000
2,2021,5200100,0.0,6.0,0.0,0.0,0.000000
3,2021,5200134,0.0,243.0,110.0,0.0,0.000000
4,2021,5200308,147.0,684.0,386.0,116.0,0.104027
...,...,...,...,...,...,...,...
17638,2014,2929206,295.0,295.0,42.0,42.0,1.000000
17639,2014,4218202,0.0,253.0,242.0,0.0,0.000000
17640,2014,1717503,13.0,13.0,34.0,34.0,1.000000
17641,2014,3540002,425.0,425.0,389.0,389.0,1.000000


In [374]:
df_clean_09['PP_ATENDIMENTO_IND'] = (df_clean_09['PP_ATENDIMENTO_IND']*100)
df_clean_09

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,QT_PUBLICA,QT_TOTAL,QT_TOTAL_2013,QT_PUBLICA_2013,PP_ATENDIMENTO_IND
0,2021,5300108,45158.0,221481.0,203821.0,32582.0,71.211778
1,2021,5200050,0.0,11.0,0.0,0.0,0.000000
2,2021,5200100,0.0,6.0,0.0,0.0,0.000000
3,2021,5200134,0.0,243.0,110.0,0.0,0.000000
4,2021,5200308,147.0,684.0,386.0,116.0,10.402685
...,...,...,...,...,...,...,...
17638,2014,2929206,295.0,295.0,42.0,42.0,100.000000
17639,2014,4218202,0.0,253.0,242.0,0.0,0.000000
17640,2014,1717503,13.0,13.0,34.0,34.0,100.000000
17641,2014,3540002,425.0,425.0,389.0,389.0,100.000000


In [375]:
df_clean_09['PP_ATENDIMENTO_IND'] = df_clean_09['PP_ATENDIMENTO_IND'].round(2)
df_clean_09

,NU_ANO_CENSO,FK_MUNICIPIO_CODIGO,QT_PUBLICA,QT_TOTAL,QT_TOTAL_2013,QT_PUBLICA_2013,PP_ATENDIMENTO_IND
0,2021,5300108,45158.0,221481.0,203821.0,32582.0,71.21
1,2021,5200050,0.0,11.0,0.0,0.0,0.00
2,2021,5200100,0.0,6.0,0.0,0.0,0.00
3,2021,5200134,0.0,243.0,110.0,0.0,0.00
4,2021,5200308,147.0,684.0,386.0,116.0,10.40
...,...,...,...,...,...,...,...
17638,2014,2929206,295.0,295.0,42.0,42.0,100.00
17639,2014,4218202,0.0,253.0,242.0,0.0,0.00
17640,2014,1717503,13.0,13.0,34.0,34.0,100.00
17641,2014,3540002,425.0,425.0,389.0,389.0,100.00


Criar coluna INDICADOR, renomear colunas e resetar index

In [376]:
df_clean_10 = df_clean_09.copy()
df_clean_10['INDICADOR'] = "12C"
df_clean_10.rename(columns={'NU_ANO_CENSO': 'ANO'}, inplace=True)
df_clean_10.reset_index(drop=True, inplace=True)
df_clean_10

,ANO,FK_MUNICIPIO_CODIGO,QT_PUBLICA,QT_TOTAL,QT_TOTAL_2013,QT_PUBLICA_2013,PP_ATENDIMENTO_IND,INDICADOR
0,2021,5300108,45158.0,221481.0,203821.0,32582.0,71.21,12C
1,2021,5200050,0.0,11.0,0.0,0.0,0.00,12C
2,2021,5200100,0.0,6.0,0.0,0.0,0.00,12C
3,2021,5200134,0.0,243.0,110.0,0.0,0.00,12C
4,2021,5200308,147.0,684.0,386.0,116.0,10.40,12C
...,...,...,...,...,...,...,...,...
17638,2014,2929206,295.0,295.0,42.0,42.0,100.00,12C
17639,2014,4218202,0.0,253.0,242.0,0.0,0.00,12C
17640,2014,1717503,13.0,13.0,34.0,34.0,100.00,12C
17641,2014,3540002,425.0,425.0,389.0,389.0,100.00,12C


</br>

</br>

</br>

---

## Pandas Method Chaining - Tutorial Intensivo

### Series e DataFrame - Semelhanças e Diferenças dos atributos

In [377]:
def what(func):
    return [x for x in dir(func) if not x[0]=="_" ]

series_methods = what(pd.Series())
df_methods = what(pd.DataFrame())

In [378]:
def show_result(list_, label):
    print(label, f'count: {len(list_)}', ', '.join(list_), sep='\n')

In [379]:
exclusive_series = [x for x in series_methods if x not in df_methods]
show_result(exclusive_series, 'exclusive_series')

exclusive_series
count: 26
argmax, argmin, argsort, array, autocorr, between, divmod, dtype, factorize, hasnans, is_monotonic_decreasing, is_monotonic_increasing, is_unique, item, map, name, nbytes, ravel, rdivmod, repeat, searchsorted, str, to_frame, to_list, unique, view


In [380]:
exclusive_df = [x for x in df_methods if x not in series_methods]
show_result(exclusive_df, 'exclusive_df')

exclusive_df
count: 31
applymap, assign, boxplot, columns, corrwith, eval, from_dict, from_records, insert, isetitem, iterrows, itertuples, join, melt, merge, pivot, pivot_table, query, select_dtypes, set_index, sparse, stack, style, to_feather, to_gbq, to_html, to_orc, to_parquet, to_records, to_stata, to_xml


In [381]:
# union series & df 
union_df_series = [x for x in df_methods if x in series_methods]
show_result(union_df_series, 'union_df_series')

union_df_series
count: 177
T, abs, add, add_prefix, add_suffix, agg, aggregate, align, all, any, apply, asfreq, asof, astype, at, at_time, attrs, axes, backfill, between_time, bfill, bool, clip, combine, combine_first, compare, convert_dtypes, copy, corr, count, cov, cummax, cummin, cumprod, cumsum, describe, diff, div, divide, dot, drop, drop_duplicates, droplevel, dropna, dtypes, duplicated, empty, eq, equals, ewm, expanding, explode, ffill, fillna, filter, first, first_valid_index, flags, floordiv, ge, get, groupby, gt, head, hist, iat, idxmax, idxmin, iloc, index, infer_objects, info, interpolate, isin, isna, isnull, items, keys, kurt, kurtosis, last, last_valid_index, le, loc, lt, mask, max, mean, median, memory_usage, min, mod, mode, mul, multiply, ndim, ne, nlargest, notna, notnull, nsmallest, nunique, pad, pct_change, pipe, plot, pop, pow, prod, product, quantile, radd, rank, rdiv, reindex, reindex_like, rename, rename_axis, reorder_levels, replace, resample, reset_index, rfloo

### Compreensão de lista e dicionários

In [382]:
# criacao de list com list comprehension
[j for j in range(21) if j%2==0]

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

In [383]:
# criacao de dicionário com dict compreension
{k:'com certeza é' for k in ['eh_exemplo','eh_string','eh_string2','eh_string3']}

{'eh_exemplo': 'com certeza é',
 'eh_string': 'com certeza é',
 'eh_string2': 'com certeza é',
 'eh_string3': 'com certeza é'}

### Criar colunas

In [384]:
(pd.DataFrame(data=range(1, 10), columns=['linhas'])
 # .assign(nova_coluna='texto')
 # .assign(nova_coluna2='texto2',
 #         outra_coluna='outro_texto')

 # .assign(**{'eh_exemplo':'com certeza é', 
 #            'eh_str':'com certeza é', 
 #            'eh_str2':'com certeza é', 
 #            'eh_str3':'com certeza é'})
 # .assign(**{k:'com certeza é' for k in ['eh_exemplo','eh_str','eh_str2','eh_str3']})

 # # criar colunas boleanas a partir de coluna existente
 # .assign(numeros=range(1,10))
 # .assign(menores_que_cinco='nao')
 # .assign(menores_que_cinco = lambda x:x['menores_que_cinco'].where(x['numeros']<5, 'sim'))

 .assign(potencia_de_dois=lambda x:x['linhas']**2)
 .assign(soma_com_dez=lambda x:x['linhas']+10)
 .assign(expressao=lambda x:x['linhas']**2+x['linhas']+10)
)

,linhas,potencia_de_dois,soma_com_dez,expressao
0,1,1,11,12
1,2,4,12,16
2,3,9,13,22
3,4,16,14,30
4,5,25,15,40
5,6,36,16,52
6,7,49,17,66
7,8,64,18,82
8,9,81,19,100


### Corrigir tipos de dados (dtypes)

In [385]:
import numpy as np

dtypes numéricos suportados pelo numpy e Pandas. Adicionalmente o Pandas suporta o dtype Int64 (com i maiúsculo), que permite armazenar valores vazios.

In [386]:
for dtype in [np.uint8, np.uint16, np.uint32, np.uint64, 
              np.int8, np.int16, np.int32, np.int64]:
    print(dtype, np.iinfo(dtype).min, ' até ', np.iinfo(dtype).max)

for dtype in [np.float16, np.float32, np.float64]:
    print(dtype, np.finfo(dtype).min, ' até ', np.finfo(dtype).max)

<class 'numpy.uint8'> 0  até  255
<class 'numpy.uint16'> 0  até  65535
<class 'numpy.uint32'> 0  até  4294967295
<class 'numpy.uint64'> 0  até  18446744073709551615
<class 'numpy.int8'> -128  até  127
<class 'numpy.int16'> -32768  até  32767
<class 'numpy.int32'> -2147483648  até  2147483647
<class 'numpy.int64'> -9223372036854775808  até  9223372036854775807
<class 'numpy.float16'> -65500.0  até  65500.0
<class 'numpy.float32'> -3.4028235e+38  até  3.4028235e+38
<class 'numpy.float64'> -1.7976931348623157e+308  até  1.7976931348623157e+308


In [387]:
(pd.DataFrame(data=[1.0, 2.0, 3.0], columns=['n1'])
 .assign(n2=[1.0, 2.0, 3.0])
 .assign(n3=lambda x:x['n2']*12345)
 # .astype({'n1':'uint8'})
 # .astype({'n3':'uint16'})
 # .astype({k:'uint8' for k in ['n1', 'n2']})
 # .info()
)

,n1,n2,n3
0,1.0,1.0,12345.0
1,2.0,2.0,24690.0
2,3.0,3.0,37035.0


### Editar múltiplas colunas com a mesma função

útil para preencher valores vazios, mudar dtype, converter valores...

In [413]:
def conversor(df, k):
    return df[k].replace({1:'um', 2:'dois'})
    
(pd.DataFrame()
 .assign(numeros_a=[1,2,3]*3,
         numeros_b=range(9,0,-1))
 # .assign(numeros_a=lambda df: conversor(df, 'numeros_a'))
 # .assign(numeros_a=lambda df: df['numeros_a'].replace({1:'um', 2:'dois'}))

 # .assign(**{k:lambda tb,  oi=k: conversor(tb, oi) for k in ['numeros_a', 'numeros_b']})
 .assign(**{k:lambda df_temp, var_temp=k: df_temp[var_temp].replace({1:'um', 2:'dois'}) 
                                   for k in ['numeros_a','numeros_b']})
)

,numeros_a,numeros_b
0,um,9
1,dois,8
2,3,7
3,um,6
4,dois,5
5,3,4
6,um,3
7,dois,dois
8,3,um


### Filtro de dados

In [389]:
(pd.DataFrame()
 .assign(numeros=[1,2,3]*3,
         letras=['a','b','c','d']*2+['e']
        )
 # .query('numeros==1')
 # .query('numeros==1 and letras=="c"')
 # .query('numeros==1 & letras=="c"')
 
 # .query('numeros==1 or letras=="c"')
 # .query('numeros==1 | letras=="c"')
 
 # .query('numeros==1 or letras!="c"')
 # .query('numeros==1 or ~(letras=="c")')
)

,numeros,letras
0,1,a
1,2,b
2,3,c
3,1,d
4,2,a
5,3,b
6,1,c
7,2,d
8,3,e


### Transformar valores em colunas - Preenchimento por contagem ou soma.

contagem de "numeros" por letra

somatório de "numeros" por letra

In [390]:
df_pivot = \
    (pd.DataFrame()
     .assign(numeros=[1,2,3,1,2,3,4,5,6,1],
             letras=['a','b']*4+['a']*2,
             nomes=['pedro','tiago']*5
            )
    )
df_pivot

,numeros,letras,nomes
0,1,a,pedro
1,2,b,tiago
2,3,a,pedro
3,1,b,tiago
4,2,a,pedro
5,3,b,tiago
6,4,a,pedro
7,5,b,tiago
8,6,a,pedro
9,1,a,tiago


Groupby

In [391]:
# groupby contagem-modo 1
(df_pivot
 .groupby(['nomes','letras'])['nomes'].count() #series 
 .unstack(-1, fill_value=0)
 .reset_index()
 .rename_axis(None, axis=1)
)

,nomes,a,b
0,pedro,5,0
1,tiago,1,4


In [392]:
# groupby contagem - modo 2
(df_pivot
 .groupby(['nomes','letras']).agg({'letras': ['count']})
 .unstack(-1, fill_value=0)
 .droplevel(level=[0,1], axis=1)
 .reset_index()
 .rename_axis(None, axis=1)
)

,nomes,a,b
0,pedro,5,0
1,tiago,1,4


In [393]:
# groupby soma
(df_pivot
 .groupby(['nomes', 'letras'])['numeros'].sum()
 .unstack(-1, fill_value=0)
 .reset_index()
 .rename_axis(None, axis=1)
)

,nomes,a,b
0,pedro,16,0
1,tiago,1,11


pivot table

In [394]:
# contagem
(df_pivot
 .pivot_table(index=['nomes'],
              columns=['letras'],
              values=['numeros'],
              aggfunc='count',
              fill_value=0
             )
 .droplevel(level=[0], axis=1)
 .reset_index()
 .rename_axis(None, axis=1)
)

,nomes,a,b
0,pedro,5,0
1,tiago,1,4


In [395]:
# soma
(df_pivot
 .pivot_table(index=['nomes'],
              columns=['letras'],
              values=['numeros'],
              aggfunc=sum,
              fill_value=0)
 .droplevel(level=[0], axis=1)
 .reset_index()
 .rename_axis(None, axis=1)
)

,nomes,a,b
0,pedro,16,0
1,tiago,1,11


### Mesclar dataframe a partir do último estado do dataframe com adaptação

criar coluna valor_base_2020 com o valor correspondente a cada 'nome' no ano 2020

.pipe: Método que aplica funções encadeadas que recebem series ou dataframes

In [396]:
# df_mesclagem = 
# [['pedro',2020,200],['pedro',2021,300], 
#                              ['tiago',2020,210],['tiago',2021,360],
#                              ['joao',2020,260],['tiago',2021,310],], 
#                              columns=['nome','ano','valor']
df_merge_temp = pd.DataFrame()
df_merge_temp

""


In [397]:
temp = (df_merge_temp
 .assign(nome=['pedro','pedro','tiago','tiago','joao','joao'])
 .assign(ano=[2020,2021,2020,2021,2020,2021])
 .assign(valor=[200,300,210,360,260,310])
)

(temp.loc[lambda y:y['ano'].isin([2020]), ['nome', 'valor']]
     .rename(columns={'valor':'valor_base_2020'})
)

,nome,valor_base_2020
0,pedro,200
2,tiago,210
4,joao,260


In [398]:
(df_merge_temp
 .assign(nome=['pedro','pedro','tiago','tiago','joao','joao'])
 .assign(ano=[2020,2021,2020,2021,2020,2021])
 .assign(valor=[200,300,210,360,260,310])
 .merge(temp.loc[lambda y:y['ano'].isin([2020]), ['nome', 'valor']]
            .rename(columns={'valor':'valor_base_2020'}),
        on='nome')
)

,nome,ano,valor,valor_base_2020
0,pedro,2020,200,200
1,pedro,2021,300,200
2,tiago,2020,210,210
3,tiago,2021,360,210
4,joao,2020,260,260
5,joao,2021,310,260


In [399]:
(df_merge_temp
 .assign(nome=['pedro','pedro','tiago','tiago','joao','joao'])
 .assign(ano=[2020,2021,2020,2021,2020,2021])
 .assign(valor=[200,300,210,360,260,310])
 .pipe(lambda w:w.merge(w.loc[lambda y:y['ano'].isin([2020]), ['nome', 'valor']]
                         .rename(columns={'valor':'valor_base_2020'}),
                        on='nome'
                       )
      )
)

,nome,ano,valor,valor_base_2020
0,pedro,2020,200,200
1,pedro,2021,300,200
2,tiago,2020,210,210
3,tiago,2021,360,210
4,joao,2020,260,260
5,joao,2021,310,260


## Pandas Method Chaining - Estudo de Caso

> **Regra de negócio**

Indicador do Plano Nacional de Educação (PNE)

**12C-Participação do segmento público na expansão de matrículas de graduação**

Fórmula: "Expansão Pública" / "Expansão Total"

Numerador: Variação das matrículas em cursos de graduação de IES públicas 

Denominador: Variação total das matrículas em cursos de graduação. 

Fórmula com variáveis: (QT_PUBLICA - QT_PUBLICA_2013) / (QT_TOTAL - QT_TOTAL_2013) * 100

Período: 2014~2021

Granularidade: Ano & Município

Dicionário de dados
- NU_ANO_CENSO: Ano do Censo
- FK_MUNICIPIO_CODIGO: Código do município de acordo com o IBGE
- TP_NIVEL_ACADEMICO: Nível acadêmico do curso, "Graduação" corresponde ao valor 1
- TP_CATEGORIA_ADMINISTRATIVA: Categoria "Pública" corresponde aos valores [1,2,3,7]
- QT_MAT: Quantidade de matrículas no curso

Características dos dados:
- Originado da tabela de Cursos do Censo de Educação Euperior, obtido no site do INEP.
- Gradunaridade de "Ano & Cursos". Cada linha contém dados de um Curso específico
- Contém linhas com valores vazios em FK_MUNICIPIO_CODIGO

`A granularidade refere-se ao nível de detalhe ou à menor unidade de informação representada por cada linha da tabela.`

In [400]:
print(df.shape)
print(df[['NU_ANO_CENSO']].value_counts())

(1831958, 5)
NU_ANO_CENSO
2021            444786
2020            335629
2019            246081
2018            177647
2017            116036
2016             89684
2015             77897
2014             70179
2013             62542
2012             60038
2011             53948
2010             50025
2009             47466
Name: count, dtype: int64


In [416]:
%%time
(df
 .dropna(subset='FK_MUNICIPIO_CODIGO', axis=0)
 .rename(columns={'NU_ANO_CENSO': 'ANO'})
 .assign(QT_MAT=lambda x:x['QT_MAT'].fillna(0))
 .astype({**{k:'uint16' for k in ['ANO','QT_MAT']}, 
          'FK_MUNICIPIO_CODIGO':'uint32',
          'TP_NIVEL_ACADEMICO':'uint8'})
 .query('TP_NIVEL_ACADEMICO==1 and ANO>=2013')
 .assign(PUBLICA="PUBLICA_SIM")
 .assign(PUBLICA=lambda x: (x['PUBLICA']
                            .where(x['TP_CATEGORIA_ADMINISTRATIVA'].isin([1,2,3,7]), 
                                   "PUBLICA_NAO")))
 .groupby(["ANO", "FK_MUNICIPIO_CODIGO","PUBLICA"])["QT_MAT"]
 .sum().unstack().reset_index().rename_axis(None, axis=1)
 .fillna(0)
 .assign(QT_TOTAL=lambda x:x['PUBLICA_SIM']+x['PUBLICA_NAO'])
 # 2013 cols
 .pipe(lambda w:(
    w.merge(w.loc[lambda y: y['ANO'].isin([2013]), 
                  ['FK_MUNICIPIO_CODIGO', 'PUBLICA_NAO', 'PUBLICA_SIM']]
             .rename(columns={'PUBLICA_SIM': 'QT_PUBLICA_2013'})
             .assign(QT_TOTAL_2013=lambda x:x['QT_PUBLICA_2013'] + x['PUBLICA_NAO'])
             
             .reindex(['FK_MUNICIPIO_CODIGO', 'QT_TOTAL_2013', 'QT_PUBLICA_2013'], 
                      axis=1), 
            on='FK_MUNICIPIO_CODIGO', how='left')))
 .rename(columns={'PUBLICA_SIM': 'QT_PUBLICA'})
 .query('ANO>=2014')
 .drop(columns=['PUBLICA_NAO'])
 .fillna(0)
 .assign(PP_ATENDIMENTO_IND = lambda x:(
     ((x['QT_PUBLICA']-x['QT_PUBLICA_2013']) / 
      (x['QT_TOTAL']-x['QT_TOTAL_2013'])
     )*100).round(2))
 .astype({**{k:'uint32' for k in ['QT_PUBLICA', 'QT_TOTAL',
                                  'QT_TOTAL_2013', 'QT_PUBLICA_2013']}})
 .assign(INDICADOR="12C")
 .reset_index(drop=True)
)

CPU times: total: 438 ms
Wall time: 428 ms


,ANO,FK_MUNICIPIO_CODIGO,QT_PUBLICA,QT_TOTAL,QT_TOTAL_2013,QT_PUBLICA_2013,PP_ATENDIMENTO_IND,INDICADOR
0,2014,1100023,434,4931,4628,333,33.33,12C
1,2014,1100049,1114,7482,6748,1156,-5.72,12C
2,2014,1100056,0,664,467,0,0.00,12C
3,2014,1100064,575,799,866,592,25.37,12C
4,2014,1100106,777,1029,928,726,50.50,12C
...,...,...,...,...,...,...,...,...
17638,2021,5221809,1106,1106,743,743,100.00,12C
17639,2021,5221858,126,9768,3216,0,1.92,12C
17640,2021,5222005,0,372,0,0,0.00,12C
17641,2021,5222054,0,1,18,0,-0.00,12C


# Mais fontes de aprendizado

- Vídeo: [Pandas Idiomático-Matt Harrison | Python Web Conf 2022](https://www.youtube.com/watch?v=Z9ekw2Ou3s0)
- Vídeo: [Pandas Efetivo-Matt Herrison | PyData Salt Lake City Meetup](https://www.youtube.com/watch?v=zgbUk90aQ6A)
- Vídeo: [Testando Pandas- Debug e otimização-Matt Herrison | PyData Global 2022](https://www.youtube.com/watch?v=Kj1WwpPFr-I)
- Compra: [Metasnake - Cursos/livros do Matt Harrison](https://www.metasnake.com/)
- Livro: [Effective Pandas - Matt Harrison](https://www.amazon.com.br/Effective-Pandas-Patterns-Data-Manipulation/dp/B09MYXXSFM/)
- Twitter: [Matt Harrison](https://twitter.com/__mharrison__)
- Vídeo: [Então você quer ser expert em pandas-James Powell | PyData Global 2021](https://www.youtube.com/watch?v=pjq3QOxl9Ok)